In [3]:
import pathlib
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import plotnine as gg
import click
import logging
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
import pandas as pd
import pathlib

In [4]:
path_customer_churn1 = pathlib.Path("../data/raw/customer_churn_0005.csv")
customer_churn1 = pd.read_csv(path_customer_churn1, delimiter=",")
len(customer_churn1)

49693

In [5]:
path_customer_usage1 = pathlib.Path("../data/raw/customer_usage_0005.csv")
customer_usage1 = pd.read_csv(path_customer_usage1, delimiter=",")
len(customer_usage1)

136302

In [6]:
path_customer_churn = pathlib.Path("../data/interim/customer_churn.csv")
path_customer_usage = pathlib.Path("../data/interim/customer_usage.csv")

In [7]:
customer_churn = pd.read_csv(path_customer_churn, delimiter=",")

In [8]:
customer_usage = pd.read_csv(path_customer_usage, delimiter=",")

In [9]:
customer_churn.describe(include="all")

,year,month,user_account_id,churn
count,5061.0,5061.0,5.061000e+03,5061.000000
mean,2013.0,9.0,5.937355e+05,0.205493
std,0.0,0.0,4.294465e+05,0.404101
min,2013.0,9.0,2.700000e+01,0.000000
25%,2013.0,9.0,2.453490e+05,0.000000
50%,2013.0,9.0,4.139200e+05,0.000000
75%,2013.0,9.0,1.041736e+06,0.000000
max,2013.0,9.0,1.196933e+06,1.000000


In [10]:
customer_usage.describe(include="all")

,year,month,user_account_id,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,...,last_100_reloads_sum,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage
count,13844.0,13844.000000,1.384400e+04,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,...,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000
mean,2013.0,7.049913,5.361070e+05,2743.831479,0.025571,42.471034,10.414656,7.306020,0.890566,0.661658,...,23.862560,109.154275,1.415873,72.973828,3.871051,112.436723,8.454565,68.706082,4.302875,9.008170
std,0.0,0.823888,4.056738e+05,4846.125488,0.157856,216.267461,14.132944,14.187962,0.312194,0.473162,...,41.177729,206.398618,8.215050,163.907837,19.135336,255.196226,34.741915,180.168112,20.052433,131.293983
min,2013.0,6.000000,2.700000e+01,0.000000,0.000000,0.000000,0.000000,-5.580000,0.000000,0.000000,...,-7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2013.0,6.000000,2.127060e+05,445.000000,0.000000,1.000000,0.890000,0.150000,1.000000,0.000000,...,0.000000,6.280000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,2013.0,7.000000,3.957340e+05,1079.000000,0.000000,1.000000,6.110000,2.785000,1.000000,1.000000,...,12.000000,41.760000,0.000000,21.080000,0.000000,22.000000,0.000000,8.000000,0.000000,0.000000
75%,2013.0,8.000000,9.868768e+05,1963.000000,0.000000,5.000000,13.890000,8.550000,1.000000,1.000000,...,30.000000,126.327500,0.000000,78.180000,0.000000,112.000000,2.000000,60.000000,0.000000,0.000000
max,2013.0,8.000000,1.196933e+06,15947.000000,1.000000,1307.000000,218.350000,317.750000,1.000000,1.000000,...,941.060000,4129.250000,348.950000,3956.200000,984.370000,3970.000000,934.000000,3861.000000,764.000000,7671.690000


In [11]:
customer_usage1.describe(include="all")


,year,month,user_account_id,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,...,last_100_reloads_sum,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage
count,136302.0,136302.000000,1.363020e+05,136302.000000,136302.000000,136302.000000,136302.000000,136302.000000,136302.000000,136302.000000,...,136302.000000,136302.000000,136302.000000,136302.000000,136302.000000,136302.000000,136302.000000,136302.000000,136302.000000,136302.000000
mean,2013.0,7.048385,5.401007e+05,2808.982099,0.025730,43.911234,10.495667,7.130304,0.883369,0.657349,...,23.272416,104.533225,1.476822,68.911207,4.232884,111.244464,7.885020,68.123681,4.240429,7.514384
std,0.0,0.823674,4.066362e+05,4940.816378,0.158328,219.491101,18.083818,14.185370,0.320981,0.474598,...,41.929064,191.389228,9.025366,144.453512,23.673347,262.471140,33.997725,186.389024,20.045359,100.722195
min,2013.0,6.000000,1.300000e+01,0.000000,0.000000,0.000000,0.000000,-30.060000,0.000000,0.000000,...,-25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2013.0,6.000000,2.163620e+05,433.000000,0.000000,1.000000,0.842500,0.120000,1.000000,0.000000,...,0.000000,5.850000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,2013.0,7.000000,3.970520e+05,1057.500000,0.000000,2.000000,6.080000,2.740000,1.000000,1.000000,...,12.000000,40.300000,0.000000,20.630000,0.000000,21.000000,0.000000,8.000000,0.000000,0.000000
75%,2013.0,8.000000,9.866910e+05,1957.000000,0.000000,5.000000,13.790000,8.460000,1.000000,1.000000,...,30.000000,123.215000,0.000000,75.777500,0.000000,112.000000,2.000000,60.000000,0.000000,0.000000
max,2013.0,8.000000,1.196944e+06,15947.000000,1.000000,1307.000000,1791.680000,1236.470000,1.000000,1.000000,...,2723.845000,6734.930000,515.080000,6500.000000,1630.670000,7254.000000,1298.000000,7230.000000,1267.000000,7671.690000


In [29]:
customer_churn1.describe(include="all")

,year,month,user_account_id,churn
count,49693.0,49693.0,4.969300e+04,49693.000000
mean,2013.0,9.0,5.957936e+05,0.208983
std,0.0,0.0,4.292740e+05,0.406586
min,2013.0,9.0,1.300000e+01,0.000000
25%,2013.0,9.0,2.518630e+05,0.000000
50%,2013.0,9.0,4.138740e+05,0.000000
75%,2013.0,9.0,1.041626e+06,0.000000
max,2013.0,9.0,1.196944e+06,1.000000


In [46]:
corr_usage1 = customer_usage1.corr(method ='pearson') 
specific_dir = "../data/output"
corr_usage1.to_excel(specific_dir+'/VariablesCorr.xlsx', sheet_name='customer_usage_corr')

In [95]:
for col in customer_usage1.columns:
    path = "../reports/figures/" + col + ".png"
    q1 = customer_usage1.quantile(.05)
    q2 = customer_usage1.quantile(.95)
    
    
    hist = gg.ggplot(customer_usage1, gg.aes(x=col)) + gg.geom_histogram() + gg.scales.xlim(q1, q2)
    #print(age_hist + gg.theme_bw())
    hist.save(path)
 

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [79]:
q1 = customer_usage1.quantile(.05)
q2 = customer_usage1.quantile(.95)
age_hist = gg.ggplot(customer_usage1, gg.aes(x="user_spendings")) + gg.geom_histogram()+
gg.scales.xlim(q1.user_spendings,q2.user_spendings) + plotnine.scales.ylim(min.user_spendings,max.user_spendings)

print(age_hist + gg.theme_bw())
path = "../reports/figures/" + col + ".png"
age_hist.save(path, width=20, height=6)


SyntaxError: invalid syntax (<ipython-input-79-61be19e1a03f>, line 3)

In [56]:
q1 = customer_usage1.quantile(.05)
print(q1.user_spendings)

0.0


In [32]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pylab
from pylab import xticks

In [90]:
q1 = customer_usage1.quantile(.05)
q2 = customer_usage1.quantile(.95)
print(q1)
   

year                                      2013.00
month                                        6.00
user_account_id                          21179.05
user_lifetime                               91.00
user_intake                                  0.00
                                           ...   
last_100_sms_outgoing_count                  0.00
last_100_sms_outgoing_to_onnet_count         0.00
last_100_sms_outgoing_to_offnet_count        0.00
last_100_sms_outgoing_to_abroad_count        0.00
last_100_gprs_usage                          0.00
Name: 0.05, Length: 65, dtype: float64
